In [37]:
# Install neccesary packages 

import pandas as pd 
import numpy as np 
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor

In [38]:
# Read file 
data = pd.read_csv("Top_250_Movie.csv")

In [39]:
# Creating a column Views to update the given value 
data["views"] = ""
data.columns

Index(['ID', 'Title', 'Year', 'Rating ', 'Rating_count', 'views'], dtype='object')

In [40]:
# Update the given values to the new columns \

data.loc[data['Title'] == 'Forest Gump', 'views'] = 10000000
data.loc[data['Title'] == 'The Usual Suspects', 'views'] = 7500000
data.loc[data['Title'] == 'Rear Window', 'views'] = 6000000
data.loc[data['Title'] == 'North by Northwest', 'views'] = 4000000
data.loc[data['Title'] == 'The Secret in Their Eyes', 'views'] = 3000000
data.loc[data['Title'] == 'Spotlight', 'views'] = 1000000


In [41]:
# Preprocessing the data 

df = data.replace(',', '', regex = True) # Removing the commas to make str object to int
nan_value = float("NaN")
df.replace("", nan_value, inplace=True) # Replacing Nan values to empty cells 

In [42]:
# Create a seperate Dataframe to store values which has Nan
non_nan_df = df[df.isna().any(axis=1)] # Nan dataframe to predict

# Excluding the Nan values 
new = df.dropna() 

In [43]:
y = new['views']  # Dependant Variable  
x = new.drop(['ID', 'Title', 'Year', 'views'], axis=1) # Independant variables


In [44]:
# changing type to numeric 
x = x.apply(pd.to_numeric, errors='ignore')

In [45]:
# applying linear regression 
reg = LinearRegression().fit(x, y)

xg_reg = xgb.XGBRegressor(objective ='reg:linear', n_estimators=1)
xg_reg.fit(x,y)

regr = RandomForestRegressor(max_depth=2, random_state=0)
regr.fit(x, y)

[23:53:33] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


RandomForestRegressor(max_depth=2, random_state=0)

In [46]:
non_nan_df['ID']

0      tt0111161
1      tt0068646
2      tt0071562
3      tt0468569
4      tt0050083
         ...    
245    tt0070047
246    tt0111495
247    tt0094625
248    tt1954470
249    tt2338151
Name: ID, Length: 245, dtype: object

In [62]:
Id_list = non_nan_df['ID'].to_list()

In [63]:
# df['views'].fillna(0, inplace=True)

In [64]:
# Storing id of movies to estimate it views 


df_pred = pd.DataFrame(columns = ['ID','Title','Year', 'Rating', 'Rating_count', 'views', 'XG_Boost_views','Linear_reg_views','Random_Forest_views'])
for i in Id_list:
    test1 = non_nan_df[(non_nan_df['ID'] == i)]
    test = test1[['Rating ', 'Rating_count']].apply(pd.to_numeric, errors='ignore')
    test1['XG_Boost_views'] = int(xg_reg.predict(test))
    test1['Linear_reg_views'] = int(reg.predict(test))
    test1['Random_Forest_views'] = int(regr.predict(test))

    df_pred.loc[i] = test1.values[0]


/var/folders/k8/n2zsd8r164l74hwq2bvv9q8m0000gp/T/ipykernel_97342/4117512741.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test1['XG_Boost_views'] = int(xg_reg.predict(test))
/var/folders/k8/n2zsd8r164l74hwq2bvv9q8m0000gp/T/ipykernel_97342/4117512741.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test1['Linear_reg_views'] = int(reg.predict(test))
/var/folders/k8/n2zsd8r164l74hwq2bvv9q8m0000gp/T/ipykernel_97342/4117512741.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

In [66]:
df_pred.to_csv("Pred.csv")

,ID,Title,Year,Rating,Rating_count,views,XG_Boost_views,Linear_reg_views,Random_Forest_views
